In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import sys
sys.executable

import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
base_dir = 'fruit_photos'
IMAGE_SIZE = 224
BATCH_SIZE = 64

data_generator = tf.keras.preprocessing.image.ImageDataGenerator(base_dir, rescale=1./255, validation_split=0.20)

## Extract and Preprocess all the data

In [3]:
train_gen = data_generator.flow_from_directory(base_dir,
                                               target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                               batch_size=BATCH_SIZE,
                                               subset='training')
val_gen = data_generator.flow_from_directory(base_dir,
                                             target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                             batch_size=BATCH_SIZE,
                                             subset='validation')

Found 56445 images belonging to 15 classes.
Found 14104 images belonging to 15 classes.


In [6]:
for image_batch, label_batch in train_gen:
    break
image_batch.shape, label_batch.shape # The shape of input tensors

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


{'Apple': 0, 'Banana': 1, 'Carambola': 2, 'Guava': 3, 'Kiwi': 4, 'Mango': 5, 'Orange': 6, 'Peach': 7, 'Pear': 8, 'Persimmon': 9, 'Pitaya': 10, 'Plum': 11, 'Pomegranate': 12, 'Tomatoes': 13, 'muskmelon': 14}


In [8]:
#Create a labels file to be exported into TFLite
print(train_gen.class_indices) # The lables
#Format them
labels = '\n'.join(train_gen.class_indices.keys())

with open('labels.txt', 'w') as f:
    f.write(labels)

{'Apple': 0, 'Banana': 1, 'Carambola': 2, 'Guava': 3, 'Kiwi': 4, 'Mango': 5, 'Orange': 6, 'Peach': 7, 'Pear': 8, 'Persimmon': 9, 'Pitaya': 10, 'Plum': 11, 'Pomegranate': 12, 'Tomatoes': 13, 'muskmelon': 14}


In [9]:
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMAGE_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

9412608/9406464 [==============================] - 3s 0us/step


In [10]:
# Freeze the base model and fine tune by stacking layers on top
base_model.trainable = False

In [11]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(32, 2, activation='relu'),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(15, activation='softmax')
])

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adagrad(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d (Conv2D)              (None, 6, 6, 32)          163872    
_________________________________________________________________
dropout (Dropout)            (None, 6, 6, 32)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 15)                495       
Total params: 2,422,351
Trainable params: 164,367
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
epochs = 15

history = model.fit_generator(train_gen,
                              epochs=epochs,
                              verbose=1,
                              validation_data=val_gen)

Epoch 1/15


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


882/882 [==============================] - 8714s 10s/step - loss: 0.5998 - accuracy: 0.8487 - val_loss: 1.8685 - val_accuracy: 0.4802
Epoch 2/15
882/882 [==============================] - 8609s 10s/step - loss: 0.2313 - accuracy: 0.9441 - val_loss: 1.7979 - val_accuracy: 0.5149
Epoch 3/15
882/882 [==============================] - 8983s 10s/step - loss: 0.1733 - accuracy: 0.9559 - val_loss: 1.8673 - val_accuracy: 0.5189
Epoch 4/15
882/882 [==============================] - 8480s 10s/step - loss: 0.1442 - accuracy: 0.9624 - val_loss: 1.8700 - val_accuracy: 0.5213
Epoch 5/15
882/882 [==============================] - 8363s 9s/step - loss: 0.1272 - accuracy: 0.9670 - val_loss: 1.8651 - val_accuracy: 0.5303
Epoch 6/15
882/882 [==============================] - 8363s 9s/step - loss: 0.1129 - accuracy: 0.9699 - val_loss: 1.9100 - val_accuracy: 0.5318
Epoch 7/15
881/882 [============================>.] - ETA: 8s - loss: 0.1056 - accuracy: 0.9721 